<a href="https://colab.research.google.com/github/aashish2000/Emotion-Detection/blob/master/Simple%20Neural%20Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization, Conv2D
from keras.preprocessing.image import ImageDataGenerator

batch_size=32
momentum=0.9
epochs=5
epoch_size=400
lr=0.005
wd=0.0005

train_datagen = ImageDataGenerator(rescale=1/255, rotation_range=90)
validation_datagen = ImageDataGenerator(rescale=1/255, rotation_range=90)
test_datagen = ImageDataGenerator(rescale=1./255,rotation_range=90)
nb_train_samples = 6186
nb_validation_samples = 1195

Using TensorFlow backend.


In [3]:
train_generator = train_datagen.flow_from_directory(
        "/content/drive/My Drive/fer_images/Training",  # This is the source directory for training images
        target_size=(48, 48),  # All images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
        "/content/drive/My Drive/fer_images/PrivateTest",  # This is the source directory for training images
        target_size=(48, 48),  # All images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)
test_generator = train_datagen.flow_from_directory(
        "/content/drive/My Drive/fer_images/PublicTest",  # This is the source directory for training images
        target_size=(48, 48),  # All images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)
filenames = test_generator.filenames
nb_test_samples = len(filenames)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
